# <u>Neural/AI Search applied to Proxy Logs</u>

The intention of this Jupyter notebook is to examine how we might apply AI search to security logs.  We'll be using fabricated proxy logs that (in theory) show benign behavior, malicious behavior, and a 950/50 mix of those two. I hope to demonstrate the following three examples:
- Semantic matching: Compare incoming logs to a vector DB of known malicious logs, flagging matches for alert
- Anomaly detection: Compare incoming logs to a vector DB of known benign logs, flagging possible anomalies for alert
- Full-text Clustering: Show clustering done with full text by leveraging vector embeddings

We'll be using a simple development-grade vector datastore called ChromaDB, and a locally-running HuggingFace LLM (large language model) that is a common one of calculating generic vector embeddings of text.

Note that there are many synonyms for <b>vector search</b>...here's a list:
- Vector similarity search
- Semantic similarity search
- Neural search (because of it's basis in neural network models for the embeddings)
- AI search
- KNN search (K-Nearest Neighbors...this is the old and/or conceptual name)
- ANN search (Approximate Nearest Neighbors) 
- HNSW (Hierarchical Navigable Small World) search (the most popular approximation algorithm)
- Probably a few others I've forgotten

All of these synonyms are referring to the same concepts and technology. It's worth noting that vector search technology has been used in large scale recommendation systems, fraud detection systems, and other "big data" types of applications for nearly a decade. In fact, handling very large volumes of full-text data is exactly what this technology was invented to handle. It's also worth noting that vector embeddings are not just relevant to text...given a suitable ML model, vector embeddings can be captured for images, audio, video, even binary executables and encrypted data. 

In [2]:
import os, sys
import pandas as pd 
import chromadb
#from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# <u>Semantic Matching: Match incoming logs again known malicious logs</u>
## Step 1: Hydrate vector DB of malicious logs
We can use this vectorstore of known malicious logs as reference data to compare incoming logs against to determine if any of those events are similar enough to raise an alert.  

In [3]:
# This CSV file contains fabricated proxy logs that are examples of malicious activity attempts
df = pd.read_csv('proxy_logs_malicious.csv')
df.columns

Index(['ID', 'IP Address', 'Timestamp', 'Log Entry'], dtype='object')

In [5]:
df.sample(3) # I've purposefully generated these to look like something that might be extracted from Splunk

,ID,IP Address,Timestamp,Log Entry
639,640,242.67.66.47,01/Aug/2024:00:58:41,"GET http://www.example.com/admin HTTP/1.1"" 500..."
478,479,98.85.143.188,01/Aug/2024:23:18:24,GET http://www.example.com/../../etc/passwd HT...
257,258,237.176.8.233,01/Aug/2024:20:29:12,GET http://www.example.com/login.php?user=admi...


In [7]:
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()
#client = chromadb.PersistentClient(path='./chromadb_proxy_logs')  # A bug prevents this from working

In [8]:
# Create collection. get_collection, get_or_create_collection, delete_collection also available
# ChromaDB uses L2 (Euclidean distance) by default...we want Cosine metric.
# Cosine similarity -> higher = better
collection = client.get_or_create_collection(name='malicious_proxy_logs', metadata={"hnsw:space": "cosine"})

In [9]:
# Create lists of the necessary data from the dataframe
ID_list = df['ID'].astype(str).tolist()  # ID list, converted to string
LogEntry_list = df['Log Entry'].tolist()   # List of documents (log content)

In [10]:

# Add docs to the collection. Can also update and delete.
# We are letting ChromaDB automatically calculate the vector embedding, instead of explicitly handling it
# By default, ChromaDB uses all-MiniLM-L6-v2 sentence transformer model to calculate vector embeddings
# This all-MiniLM-L6-v2 model provides a 384 dimension vector that can be used for embedding and clustering
collection.add(
    documents=LogEntry_list, # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    #metadatas=[{"source": "notion"}, {"source": "google-docs"}], # metadata filters
    ids=ID_list, # unique ID for each doc
)

In [ ]:
collection.count()  # Check our count to make sure it looks right

1000

In [11]:
# Examine a record by ID...tell it to show the vector embedding so we can see what it looks like
# The vector embedding is the 384 dimension numeric representation of what that text "means"...
collection.get('1', include=['embeddings', 'documents', 'metadatas'])
#collection.get(['1','2'], include=['embeddings', 'documents', 'metadatas'])

{'ids': ['1'],
 'embeddings': [array([ 7.62485852e-03,  4.53667305e-02, -4.74873595e-02,  6.38289424e-03,
          1.04712076e-01, -9.29494649e-02,  3.30195762e-03, -2.59210616e-02,
          5.31232208e-02, -9.98799801e-02, -1.34964027e-02,  2.52751410e-02,
          9.09934863e-02, -2.30148784e-03, -3.68964337e-02,  2.72639599e-02,
         -3.08614544e-05,  1.75895784e-02,  4.42130752e-02, -4.82321531e-02,
          9.13773328e-02,  3.76302861e-02,  4.21186276e-02, -8.69319122e-03,
         -7.84153715e-02, -4.20195386e-02, -3.71134132e-02, -1.43288216e-02,
          3.94372605e-02,  7.00980704e-03,  4.29793000e-02, -1.34540960e-01,
          1.49487145e-02, -1.01147126e-02,  6.74820086e-03, -2.62145475e-02,
          2.07935330e-02,  5.18131116e-03,  1.60162561e-02, -8.18299782e-03,
         -7.13483337e-03, -1.71514926e-03,  5.44389822e-02,  5.02826646e-02,
         -1.11683058e-02, -3.28837633e-02, -8.96301419e-02, -5.58557063e-02,
         -8.27203915e-02, -8.97455681e-03, -4.6

In [14]:
# Let's examine simple calcs around vector distance to foster understanding
import numpy
import math
from scipy import spatial

# Grab data from two of the records we've put into the vector DB to examine
vec1 = collection.get('1', include=['embeddings', 'documents'])['embeddings'][0]
vec2 = collection.get('2', include=['embeddings', 'documents'])['embeddings'][0]

# Calc the euclidean (or L2) distance between those two vectors
# This is the same thing you'd do with geographic distance between 2 cities
# Euclidean provides magnitude but not direction...a lower distance is better match
euclidean_dist = spatial.distance.euclidean(vec1, vec2)
print('euclidean distance: ', euclidean_dist)

# Cosine similarity is the difference between angles...it provides direction but not magnitude
# Cosine is often a good metric for text comparison...not that a higher value is better match
cosine_dist = spatial.distance.cosine(vec1, vec2)
print('cosine similarity: ', cosine_dist)


euclidean distance:  0.8742813467979431
cosine similarity:  0.38218384981155396


In [17]:
# Execute an ANN query/search for K most similar results.
results = collection.query(
    query_texts=["http://www.example.com/../../etc/passwd"], # This gets vectorized and used for vector query
    n_results=3,
    # where_document={"$contains":"Macintosh"}  # optional keyword filter
    # where={"metadata_field": "is_equal_to_this"}, # optional metadata filter
)

results

{'ids': [['660', '620', '42']],
 'distances': [[0.3781334161758423, 0.3815879225730896, 0.38311684131622314]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['GET http://www.example.com/../../etc/passwd HTTP/1.1" 500 322256 "http://www.example.com/../../etc/passwd" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
   'GET http://www.example.com/../../etc/passwd HTTP/1.1" 400 190721 "http://www.example.com/../../etc/passwd" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
   'GET http://www.example.com/index.php?page=../../../../../../etc/passwd HTTP/1.1" 400 665698 "http://www.example.com/index.php?page=../../../../../../etc/passwd" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15']],
 'uris': None,
 'data': None}

In [20]:
# Ask the same question, but this time in plain English!
results = collection.query(
    query_texts=["Can you show me possible attempts to change a password?"],  # Vectorize and search
    n_results=3,
    # where_document={"$contains":"script"}  # optional keyword filter
    # where={"metadata_field": "is_equal_to_this"}, # optional metadata filter
)

results

{'ids': [['730', '573', '153']],
 'distances': [[0.6075342893600464, 0.6118622422218323, 0.6151671409606934]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['GET http://www.example.com/index.php?page=../../../../../../etc/passwd HTTP/1.1" 403 580156 "http://www.example.com/index.php?page=../../../../../../etc/passwd" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
   'GET http://www.example.com/index.php?page=../../../../../../etc/passwd HTTP/1.1" 403 530538 "http://www.example.com/index.php?page=../../../../../../etc/passwd" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
   'GET http://www.example.com/index.php?page=../../../../../../etc/passwd HTTP/1.1" 403 190296 "http://www.example.com/index.php?page=../../../../../../etc/passwd" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, 

# Step 2: Semantic matching of malicious attempts
We have a vector DB of malicious logs to execute neural searches against.  Now, let's feed it some fresh logs to see what matches we get.  We want to use cosine similarity for this.  We'll have to experiment a bit to set a reasonable threshold for when to return an alert vs not.

In [21]:
# Some fabricated proxy logs that contain 950 benign log entries and 50 malicious log entries
df = pd.read_csv('proxy_logs_mixed.csv')

In [22]:
df.sample(3)

,ID,IP Address,Timestamp,Log Entry
703,704,184.135.136.56,01/Aug/2024:01:53:57,"GET http://cloudstorage.com/files HTTP/1.1"" 20..."
170,171,76.220.203.138,01/Aug/2024:21:53:37,"GET http://update.software.com/check HTTP/1.1""..."
681,682,168.100.131.219,01/Aug/2024:22:16:40,"GET http://www.example.com/script.js HTTP/1.1""..."


In [38]:
incoming_proxy_logs = df['Log Entry']
incoming_proxy_logs

0      GET http://www.example.com/ HTTP/1.1" 200 4516...
1      GET http://intranet.company.com/home HTTP/1.1"...
2      GET http://update.software.com/check HTTP/1.1"...
3      GET http://update.software.com/version.xml HTT...
4      GET http://www.example.com/ HTTP/1.1" 200 6339...
                             ...                        
995    GET http://www.example.com/admin HTTP/1.1" 400...
996    GET http://www.example.com/admin HTTP/1.1" 403...
997    GET http://www.example.com/../../etc/passwd HT...
998    GET http://www.example.com/admin HTTP/1.1" 403...
999    GET http://www.example.com/search?q=<script>al...
Name: Log Entry, Length: 1000, dtype: object

In [28]:
# Find incoming log entries that match known malicious activity at a pre-determined threshold 
for log_entry in incoming_proxy_logs:
    results = collection.query(
    query_texts=log_entry,
    n_results=1,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
    )
    if results['distances'][0][0] >= 0.488:  # Threshold for a match
       print(f'Possible alert on: {log_entry} \n  -- Match within threshold on {results}')

Possible alert on: GET http://cloudstorage.com/files HTTP/1.1" 200 365923 "http://cloudstorage.com/files" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 
  -- Match within threshold on {'ids': [['251']], 'distances': [[0.4886305332183838]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['GET http://www.example.com/admin HTTP/1.1" 500 108615 "http://www.example.com/admin" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15']], 'uris': None, 'data': None}
Possible alert on: GET http://cloudstorage.com/files HTTP/1.1" 200 142231 "http://cloudstorage.com/files" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 
  -- Match within threshold on {'ids': [['251']], 'distances': [[0.4881707429885864]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['GET http://www.example

# <u>Anomaly Detection by AI search on Benign Logs</u>
## Step 1: Build vector DB of Benign Proxy log data

In [29]:
# To conserve memory, re-use the previous collection for new data
collection = client.get_or_create_collection(name='benign_proxy_logs', metadata={"hnsw:space": "cosine"})

In [30]:
# This CSV file contains fabricated proxy logs that are examples of benign activity
df = pd.read_csv('proxy_logs_good.csv')
df.sample(3)

,ID,IP Address,Timestamp,Log Entry
536,537,7.167.12.102,01/Aug/2024:16:45:10,"GET http://www.example.com/style.css HTTP/1.1""..."
544,545,111.35.249.4,01/Aug/2024:07:00:10,"GET http://cloudstorage.com/files HTTP/1.1"" 20..."
764,765,219.207.141.124,01/Aug/2024:01:31:38,"GET http://www.example.com/script.js HTTP/1.1""..."


In [31]:
# Create lists of the necessary data from the dataframe
ID_list = df['ID'].astype(str).tolist()  # ID list, converted to string
LogEntry_list = df['Log Entry'].tolist()   # List of documents (log content)

In [32]:
# Add log records to vector store, allowing ChromaDB to calculate vector embeddings
collection.add(
    documents=LogEntry_list, # we handle tokenization, embedding, and indexing automatically. 
    #metadatas=[{"source": "notion"}, {"source": "google-docs"}], # metadata filters
    ids=ID_list, # unique ID for each doc
)

In [34]:
collection.count()  # Check our count to make sure it looks right

1000

In [35]:
# Check a few records...
collection.get(['1','2','3'])

{'ids': ['1', '2', '3'],
 'embeddings': None,
 'metadatas': [None, None, None],
 'documents': ['GET http://www.example.com/ HTTP/1.1" 200 140321 "http://www.example.com/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
  'GET http://update.software.com/check HTTP/1.1" 200 943292 "http://update.software.com/check" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
  'GET http://www.example.com/ HTTP/1.1" 200 813021 "http://www.example.com/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'],
 'uris': None,
 'data': None}

# Step 1: Vector search for Anomaly Detection
With a vector DB of known benign/good proxy log entries, we can do semantic comparison to flag incoming log entries that are too different from what we know to be benign logs. 

In [36]:
# We'll use fabricated proxy logs that contain 950 benign log entries and 50 malicious log entries again
df = pd.read_csv('proxy_logs_mixed.csv')
df.sample(3)

,ID,IP Address,Timestamp,Log Entry
716,717,234.213.212.197,01/Aug/2024:23:55:43,"GET http://cloudstorage.com/files HTTP/1.1"" 20..."
453,454,111.10.144.68,01/Aug/2024:05:17:11,"GET http://www.example.com/style.css HTTP/1.1""..."
85,86,60.185.34.223,01/Aug/2024:19:39:38,"GET http://www.example.com/ HTTP/1.1"" 200 7311..."


In [37]:
# Grab just the log entry iself
incoming_proxy_logs = df['Log Entry']
incoming_proxy_logs

0      GET http://www.example.com/ HTTP/1.1" 200 4516...
1      GET http://intranet.company.com/home HTTP/1.1"...
2      GET http://update.software.com/check HTTP/1.1"...
3      GET http://update.software.com/version.xml HTT...
4      GET http://www.example.com/ HTTP/1.1" 200 6339...
                             ...                        
995    GET http://www.example.com/admin HTTP/1.1" 400...
996    GET http://www.example.com/admin HTTP/1.1" 403...
997    GET http://www.example.com/../../etc/passwd HT...
998    GET http://www.example.com/admin HTTP/1.1" 403...
999    GET http://www.example.com/search?q=<script>al...
Name: Log Entry, Length: 1000, dtype: object

In [39]:
# Let's identify anomalies in the incoming logs by using vector search against DB of known benign logs
for log_entry in incoming_proxy_logs:
    results = collection.query(
    query_texts=log_entry,
    n_results=1,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
    )
    if results['distances'][0][0] <= 0.0001:  # Threshold for a non-match
       print(f'Possible anomaly on: {log_entry} \n  -- Different within threshold on {results}')

Possible anomaly on: GET http://intranet.company.com/docs/document.pdf HTTP/1.1" 200 937423 "http://intranet.company.com/docs/document.pdf" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15 
  -- Different within threshold on {'ids': [['250']], 'distances': [[9.751319885253906e-05]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['GET http://intranet.company.com/docs/document.pdf HTTP/1.1" 200 787396 "http://intranet.company.com/docs/document.pdf" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15']], 'uris': None, 'data': None}
Possible anomaly on: GET http://cloudstorage.com/authenticate HTTP/1.1" 200 596569 "http://cloudstorage.com/authenticate" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15 
  -- Different within threshold on {'ids': [['597']], 'distances': [[

# <u>Full-text Clustering with Vector Embeddings</u>
We will use the mixed proxy log data which 950 benign logs + 50 malicious logs to see if we can segregate the two using clustering. This is a notable exercise...only recently has this technology advanced to the point that we can actually do Clustering with full text such as security logs!


In [40]:
# We'll use the mixed proxy logs to see if we can get them clustered into 950 benign + 50 malicious
df = pd.read_csv('proxy_logs_mixed.csv')
df = df.drop(['IP Address', 'Timestamp'], axis=1)  # We don't need IP nor timestamp for this task
df.sample(3)


,ID,Log Entry
378,379,"GET http://cloudstorage.com/login HTTP/1.1"" 20..."
248,249,GET http://update.software.com/version.xml HTT...
117,118,GET http://cloudstorage.com/authenticate HTTP/...


In [41]:
# It's easier to just calc the vector embeddings explicitly, vs adding inserting to ChromaDB to get embeddings
# ChromaDB can't do clustering queries (yet), but we will use the same embedding model that ChromaDB is using

# Load the embedding model so we can use it to easily populate the dataframe
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# Use like this: embeddings = model.encode(whatever_text)

# Use a lambda function to encode the text in each row and apply it to a new column
df['embedding'] = df['Log Entry'].apply(lambda text:model.encode(text))
df.sample(3)

c:\Users\david\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ID,Log Entry,embedding
461,462,GET http://intranet.company.com/docs/document....,"[-0.010656089, -0.02051939, -0.058902238, -0.0..."
499,500,"GET http://cloudstorage.com/files HTTP/1.1"" 20...","[-0.019968366, 0.032466516, 0.018314362, -0.00..."
307,308,GET http://update.software.com/version.xml HTT...,"[-0.020442212, 0.005210802, 0.02564754, -0.055..."


In [42]:
# import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# Assuming you have a DataFrame `df` with columns: 'ID', 'text', and 'embedding'
# Here 'embedding' contains the 384-dimensional vectors

# Extract the embeddings from the DataFrame
embeddings = df['embedding'].tolist()

# Convert the list of embeddings into a numpy array
# import numpy as np
X = np.array(embeddings)

# Perform K-means clustering with K=2
# We know we have good logs and bad logs, so let's try to cluster them into good and bad
kmeans = KMeans(n_clusters=2, random_state=0)
df['cluster'] = kmeans.fit_predict(X)

# Now `df` will have a new column 'cluster' indicating the cluster each entry belongs to
df.columns


c:\Users\david\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Index(['ID', 'Log Entry', 'embedding', 'cluster'], dtype='object')

In [56]:
df.sample(3)

,ID,Log Entry,embedding,cluster
811,812,"GET http://www.example.com/style.css HTTP/1.1""...","[-0.013542056, 0.059399564, -0.008377618, 0.00...",1
83,84,GET http://update.software.com/version.xml HTT...,"[-0.018234754, 0.0017733873, 0.025784267, -0.0...",1
183,184,GET http://intranet.company.com/docs/document....,"[-0.0107954405, -0.019446967, -0.06237483, -0....",1


In [58]:
# This should be in the ballpark of 950 in one cluster group and 50 in the other cluster group
df['cluster'].value_counts()

cluster
1    711
0    289
Name: count, dtype: int64